In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn

import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse

## Argument Declare

In [3]:
DATASET_ROOT = "/media/arg_ws3/5E703E3A703E18EB/data/subt_artifact"
DATASET_NAME = "person"
cfg = subt
BASE_NET = "./weights/vgg16_reducedfc.pth"
DATA_DETECTION = SUBTDetection
BATCH_SIZE = 32
PRETRAINED_MODEL = None
PRETRAINED_ITER = 0
SAVE_MODEL_ITER = 500
START_ITER = 0
NUM_WORKERS = 4
CUDA = True
LR = 1e-3
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
GAMMA = 0.1
VISDOM = False
SAVE_FOLDER = "./weights/" + DATASET_NAME + "/"
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

In [4]:
if torch.cuda.is_available():
    if not CUDA:
        print("WTF are u wasting your CUDA device?")
    else:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

## Define function

In [5]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

# Initial model weights & bias
def xavier(param):
    init.xavier_uniform(param)
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

# Adjust learning rate during training
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = LR * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        print("Change learning rate to: ", lr)

## Create dataset

In [6]:
dataset = DATA_DETECTION(root=DATASET_ROOT, image_sets=['train'],transform=SSDAugmentation(cfg['min_dim'], MEANS))

classes = dataset.target_transform.class_to_ind
print("Class to index: \n", classes)
classes = sorted(classes.items(), key=lambda kv: kv[1])
label = []
for i in classes:
    label.append(i[0])
label.append('None')
print(label)
true_label = ['backpack', 'extinguisher', 'toolbox', 'radio', 'valve']

Class to index: 
 {'backpack': 0, 'toolbox': 3, 'extinguisher': 1, 'radio': 2, 'valve': 4}
['backpack', 'extinguisher', 'radio', 'toolbox', 'valve', 'None']


## Create model

In [7]:
# Delcare SSD Network
ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net
if CUDA:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
    
if PRETRAINED_MODEL is not None: # Use SSD pretrained model
    print('Resuming training, loading {}...'.format(PRETRAINED_MODEL))
    ssd_net.load_weights(SAVE_FOLDER + PRETRAINED_MODEL)
else:
    print('Initializing weights...')
    vgg_weights = torch.load(BASE_NET) # load vgg pretrained model
    ssd_net.extras.apply(weights_init) # Initial SSD model weights & bias
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM,
                weight_decay=WEIGHT_DECAY)
criterion = MultiBoxLoss(BATCH_SIZE ,cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                False, CUDA)

Initializing weights...


/home/arg_ws3/ssd.pytorch/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/arg_ws3/ssd.pytorch/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)
/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


## Training

### Initialize

In [8]:
net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
print('Loading the dataset...')
epoch_size = len(dataset) // BATCH_SIZE
print('Training SSD on:', DATASET_NAME)

data_loader = data.DataLoader(dataset, BATCH_SIZE,
                                num_workers=NUM_WORKERS,
                                shuffle=True, collate_fn=detection_collate,
                                pin_memory=True)
batch_iterator = iter(data_loader)

Loading the dataset...
Training SSD on: person


### Start training

In [8]:
step_index = 0
for iteration in range(START_ITER, cfg['max_iter']):
    if iteration in cfg['lr_steps']:
        step_index += 1
        adjust_learning_rate(optimizer, GAMMA, step_index)
    
    # make sure data iter not out of range
    try:
        images, targets = next(batch_iterator)
        #print(targets[0][0][4].item(), label[int(targets[0][0][4].item())])
    except StopIteration:
        batch_iterator = iter(data_loader)
        images, targets = next(batch_iterator)
    if CUDA:
        images = Variable(images.cuda())
        targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
    else:
        images = Variable(images)
        targets = [Variable(ann, volatile=True) for ann in targets]
    
    # Forward
    t0 = time.time()
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.item()
    conf_loss += loss_c.item()
    
    if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(PRETRAINED_ITER + iteration) + ' || Loss: %.4f ||' % (loss.item()), end='')
    
    if iteration != 0 and iteration % SAVE_MODEL_ITER == 0:
            print('Saving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), SAVE_FOLDER + DATASET_NAME + "_" +
                       repr(PRETRAINED_ITER + iteration) + '.pth')
# Save final model
torch.save(ssd_net.state_dict(),
            SAVE_FOLDER + DATASET_NAME + '.pth')

/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()
/home/arg_ws3/.local/lib/python3.5/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 14.2039 sec.
iter 0 || Loss: 16.9631 ||timer: 0.3456 sec.
iter 10 || Loss: 13.8549 ||timer: 0.3826 sec.
iter 20 || Loss: 13.9038 ||timer: 0.3843 sec.
iter 30 || Loss: 18.7738 ||timer: 0.3858 sec.
iter 40 || Loss: 16.3145 ||timer: 0.3849 sec.
iter 50 || Loss: 16.7607 ||timer: 0.3927 sec.
iter 60 || Loss: 12.4441 ||timer: 0.3766 sec.
iter 70 || Loss: 11.0996 ||timer: 0.3807 sec.
iter 80 || Loss: 17.6200 ||timer: 0.3808 sec.
iter 90 || Loss: 25.8369 ||timer: 0.3908 sec.
iter 100 || Loss: 18.1654 ||timer: 0.3792 sec.
iter 110 || Loss: 22.1077 ||timer: 0.3831 sec.
iter 120 || Loss: 16.1234 ||timer: 0.3944 sec.
iter 130 || Loss: 15.5866 ||timer: 0.3774 sec.
iter 140 || Loss: 11.0364 ||timer: 0.3880 sec.
iter 150 || Loss: 14.4047 ||timer: 0.3803 sec.
iter 160 || Loss: 10.2705 ||timer: 0.3469 sec.
iter 170 || Loss: 6.8627 ||timer: 0.3798 sec.
iter 180 || Loss: 10.6124 ||timer: 0.3892 sec.
iter 190 || Loss: 18.0428 ||timer: 0.3844 sec.
iter 200 || Loss: 12.5876 ||timer: 0.3837 sec.
iter 

iter 1740 || Loss: 3.1312 ||timer: 0.3841 sec.
iter 1750 || Loss: 2.8793 ||timer: 0.3788 sec.
iter 1760 || Loss: 3.0130 ||timer: 0.4413 sec.
iter 1770 || Loss: 3.4527 ||timer: 0.4074 sec.
iter 1780 || Loss: 3.5615 ||timer: 0.3965 sec.
iter 1790 || Loss: 3.7266 ||timer: 0.3864 sec.
iter 1800 || Loss: 3.0230 ||timer: 0.3903 sec.
iter 1810 || Loss: 2.9804 ||timer: 0.3998 sec.
iter 1820 || Loss: 2.8286 ||timer: 0.3856 sec.
iter 1830 || Loss: 2.8353 ||timer: 0.3801 sec.
iter 1840 || Loss: 3.8309 ||timer: 0.3858 sec.
iter 1850 || Loss: 3.5399 ||timer: 0.3850 sec.
iter 1860 || Loss: 3.4252 ||timer: 0.4064 sec.
iter 1870 || Loss: 4.1283 ||timer: 0.3828 sec.
iter 1880 || Loss: 3.3502 ||timer: 0.3938 sec.
iter 1890 || Loss: 4.4071 ||timer: 0.3711 sec.
iter 1900 || Loss: 2.4552 ||timer: 0.3866 sec.
iter 1910 || Loss: 2.9383 ||timer: 0.3846 sec.
iter 1920 || Loss: 4.3787 ||timer: 0.3812 sec.
iter 1930 || Loss: 3.2870 ||timer: 0.3791 sec.
iter 1940 || Loss: 3.1594 ||timer: 0.3845 sec.
iter 1950 || 

iter 3470 || Loss: 2.6593 ||timer: 0.3875 sec.
iter 3480 || Loss: 2.3717 ||timer: 0.3895 sec.
iter 3490 || Loss: 1.9139 ||timer: 0.3835 sec.
iter 3500 || Loss: 2.2740 ||Saving state, iter: 3500
timer: 0.3840 sec.
iter 3510 || Loss: 1.9930 ||timer: 0.3841 sec.
iter 3520 || Loss: 2.4525 ||timer: 0.3847 sec.
iter 3530 || Loss: 2.0850 ||timer: 0.3847 sec.
iter 3540 || Loss: 2.3230 ||timer: 0.3855 sec.
iter 3550 || Loss: 2.1369 ||timer: 0.3866 sec.
iter 3560 || Loss: 3.0490 ||timer: 0.3796 sec.
iter 3570 || Loss: 2.7265 ||timer: 0.3859 sec.
iter 3580 || Loss: 2.3639 ||timer: 0.3853 sec.
iter 3590 || Loss: 2.7483 ||timer: 0.3837 sec.
iter 3600 || Loss: 2.2525 ||timer: 0.3798 sec.
iter 3610 || Loss: 2.2670 ||timer: 0.3842 sec.
iter 3620 || Loss: 2.4710 ||timer: 0.3845 sec.
iter 3630 || Loss: 2.4113 ||timer: 0.3797 sec.
iter 3640 || Loss: 2.1086 ||timer: 0.3803 sec.
iter 3650 || Loss: 2.1652 ||timer: 0.3845 sec.
iter 3660 || Loss: 1.9295 ||timer: 0.3834 sec.
iter 3670 || Loss: 2.4068 ||timer: 

KeyboardInterrupt: 